In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math
import csv
from numpy.linalg import norm

In [7]:
# folders = (
#     "GMRES/GMRES_64_16",
#     "GMRES/GMRES_64_32",
#     "CLASSICAL_GMRES/CLASSICAL_GMRES_64_16",
#     "CLASSICAL_GMRES/CLASSICAL_GMRES_64_32",
#     "CLASSICAL/CLASSICAL_64_16",
#     "CLASSICAL/CLASSICAL_64_32",
# )

folders = (
    "GMRES",
    "CLASSICAL_GMRES",
    "CLASSICAL"
)

matrices = ("Bai/af23560", "Engwirda/airfoil_2d", 
    "Simon/appu", "vanHeukelum/cage10", 
    "Oberwolfach/chipcool1", "Averous/epb1", 
    "Botonakis/FEM_3D_thermal1", "Oberwolfach/inlet", 
    "Hollinger/jan99jac040sc", "FEMLAB/ns3Da", 
    "FEMLAB/poisson3Da",
    "Wang/wang3", "Wang/wang4", "Zhao/Zhao1", "Zhao/Zhao2"
)

In [20]:
#we'll use the median niters, store it in data.csv
#folders: GMRES, classical_gmres, classical

def clean_niters(series): 
    val = int(series.median())
    if val in (-50, -49): 
        return "--"
    if val == -5: 
        return "*"
    return val
def get_data(path): #caveat: not everything converges
    df = pd.read_csv(path)
    data = {}
    data["niters sp"] = clean_niters(df["niters_sp"])
    data["niters 64 16"] = clean_niters(df["niters_64_16"])
    data["niters 64 32"] = clean_niters(df["niters_64_32"])


    data["time sp"] = round(df["time_sp"].median(), 5) #round to 5 decimal places
    data['time 64 16'] = round(df['time_64_16'].median(), 5)
    data['time 64 32'] = round(df['time_64_32'].median(), 5)
    return data

#conv_sp,conv_64_16,conv_64_32,niters_sp,niters_64_16,niters_64_32,flag_sp,flag_64_16,flag_64_32,time_sp,time_64_16,time_64_32 


data = [] #list of dictionaries
# with open("data.csv", "w", newline='') as csvfile:
for matrix in matrices:
    row = {}
    d = np.load(os.path.join("../matrix_stats", matrix.replace("/", "_") + ".npz"))
    row["cond"] = d["cond"][0]
    row["ocond"] = "$O(10^{" + str(math.floor(math.log(d["cond"][0], 10))) + "})$"
    row["norm"] = d["norm"][0]
    row["onorm"] = "$O(10^{" + str(math.floor(math.log(d["norm"][0], 10))) + "})$"
    row["name"] = matrix
    
    for folder in folders:
        folder = "H200/" + folder
        ref = folder.split("/")[1].replace("_", " ")
        d = get_data(os.path.join(folder, matrix, "convergence_results.csv"))
        row[ref + " niters sp"] = d["niters sp"]
        row[ref + " niters 64 16"] = d["niters 64 16"]
        row[ref + " niters 64 32"] = d["niters 64 32"]

        row[ref + " time sp"] = d["time sp"]
        row[ref + " time 64 16"] = d["time 64 16"]
        row[ref + " time 64 32"] = d["time 64 32"]

    data.append(row)

with open("H200.csv", mode = "w", newline='') as file: 
    writer = csv.DictWriter(file, fieldnames = data[0].keys())
    writer.writeheader()
    writer.writerows(data)


